Este código realiza un proceso automatizado para consultar y combinar datos históricos (2000–2024) de indicadores sociales (natalidad y desnutrición) en países de Sudamérica usando la API del Banco Mundial. 

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm  # Progreso opcional

# --- Parámetros ---
sudamerica_nombres = [
    "Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador",
    "Paraguay", "Peru", "Uruguay", "Venezuela"
]

indicadores = {
    "SP.DYN.CBRT.IN": "Tasa de Natalidad (por 1000 habitantes)",
    "SN.ITK.DEFC.ZS": "Desnutrición (%)"
}
fecha = "2000:2024"

# --- Obtener lista de países disponibles ---
def obtener_paises_sudamericanos():
    url = "http://api.worldbank.org/v2/country?format=json&per_page=400"
    response = requests.get(url)
    data = response.json()
    paises = {
        pais["id"]: pais["name"]
        for pais in data[1]
        if pais["name"] in sudamerica_nombres
    }
    return paises

# --- Función para obtener datos de un país e indicador ---
def obtener_datos(pais_codigo, pais_nombre, indicador_codigo, indicador_nombre):
    url = (
        f"http://api.worldbank.org/v2/country/{pais_codigo}/indicator/{indicador_codigo}"
        f"?format=json&date={fecha}&per_page=100"
    )
    try:
        response = requests.get(url)
        data = response.json()
    except Exception:
        soup = BeautifulSoup(response.text, "html.parser")
        print(f"⚠️  Error al obtener datos para {pais_nombre} - {indicador_nombre}")
        return pd.DataFrame()

    if len(data) > 1:
        datos = [
            {
                "País": pais_nombre,
                "Año": int(d["date"]),
                "Indicador": indicador_nombre,
                "Valor": d["value"]
            }
            for d in data[1] if d["value"] is not None
        ]
        return pd.DataFrame(datos)
    else:
        return pd.DataFrame()

# --- Descarga de datos ---
def descargar_datos():
    paises = obtener_paises_sudamericanos()
    todos_los_datos = []

    print("🔄 Obteniendo datos desde la API del Banco Mundial...")
    for pais_codigo, pais_nombre in tqdm(paises.items()):
        for indicador_codigo, indicador_nombre in indicadores.items():
            df = obtener_datos(pais_codigo, pais_nombre, indicador_codigo, indicador_nombre)
            if not df.empty:
                todos_los_datos.append(df)

    if todos_los_datos:
        df_completo = pd.concat(todos_los_datos, ignore_index=True)
        return df_completo
    else:
        return pd.DataFrame()

# --- Ejecutar todo ---
df = descargar_datos()

# --- Pivotear tabla ---
if not df.empty:
    df_pivot = df.pivot_table(
        index=["Año", "País"],
        columns="Indicador",
        values="Valor"
    ).reset_index().sort_values(["País", "Año"])

    # --- Vista previa ---
    print("\n✅ Vista previa de los datos combinados:")
    print(df_pivot.head())
else:
    print("❌ No se encontraron datos disponibles.")


🔄 Obteniendo datos desde la API del Banco Mundial...


100%|██████████| 9/9 [00:04<00:00,  1.99it/s]


✅ Vista previa de los datos combinados:
Indicador   Año       País  Desnutrición (%)  \
0          2000  Argentina               NaN   
9          2001  Argentina               2.9   
18         2002  Argentina               3.7   
27         2003  Argentina               4.3   
36         2004  Argentina               3.9   

Indicador  Tasa de Natalidad (por 1000 habitantes)  
0                                           19.328  
9                                           18.940  
18                                          18.702  
27                                          18.410  
36                                          18.295  


Guardando data frama en un excel

In [4]:
df_pivot.to_excel("indicadores_sudamerica.xlsx", index=False)


Analisis grafico del indice de desnutricion por pais en los años 2001 - 2022